# Character-level text generation with LSTM

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/15<br>
**Last modified:** 2020/04/30<br>
**Description:** Generate text from Nietzsche's writings with a character-level LSTM.

## Introduction

This example demonstrates how to use a LSTM model to generate
text character-by-character.

At least 20 epochs are required before the generated text
starts sounding locally coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.


## Setup


In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import numpy as np
import random
import io




## Prepare the data


In [11]:

with io.open("/content/hello.txt", encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 10
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



Corpus length: 76787
Total chars: 6
Number of sequences: 25593


<ipython-input-11-f473aaf02d6b>:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-11-f473aaf02d6b>:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


## Build the model: a single LSTM layer


In [12]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
checkpoint_callback = ModelCheckpoint('model_weights.h5', save_weights_only=True)
csv_logger = CSVLogger('training_log.csv', append=True)
model.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics = ['accuracy'])


In [14]:
num_epochs = 100
batch_size = 200

In [15]:
model.fit(x, y,batch_size=batch_size, epochs=num_epochs, callbacks=[checkpoint_callback, csv_logger])

Epoch 1/100
128/128 [==============================] - 2s 4ms/step - loss: 1.7951 - accuracy: 0.1659
Epoch 2/100
128/128 [==============================] - 1s 4ms/step - loss: 1.7932 - accuracy: 0.1682
Epoch 3/100
128/128 [==============================] - 1s 4ms/step - loss: 1.7926 - accuracy: 0.1706
Epoch 4/100
128/128 [==============================] - 1s 5ms/step - loss: 1.7928 - accuracy: 0.1688
Epoch 5/100
128/128 [==============================] - 1s 4ms/step - loss: 1.7926 - accuracy: 0.1701
Epoch 6/100
128/128 [==============================] - 1s 4ms/step - loss: 1.7924 - accuracy: 0.1689
Epoch 7/100
128/128 [==============================] - 1s 4ms/step - loss: 1.7925 - accuracy: 0.1739
Epoch 8/100
128/128 [==============================] - 1s 4ms/step - loss: 1.7926 - accuracy: 0.1710
Epoch 9/100
128/128 [==============================] - 1s 5ms/step - loss: 1.7924 - accuracy: 0.1729
Epoch 10/100
128/128 [==============================] - 1s 6ms/step - loss: 1.7923 - accura

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

# Create a ModelCheckpoint callback to save the model weights after each epoch
checkpoint_callback = ModelCheckpoint('model_weights.h5', save_weights_only=True)

# Create a CSVLogger callback to log the loss and accuracy after each epoch
csv_logger = CSVLogger('training_log.csv', append=True)

In [ ]:
model.fit(x, y, batch_size=batch_size, epochs=e)

Epoch 1/100
320/320 [==============================] - 1s 4ms/step - loss: 0.8357 - accuracy: 0.6933
Epoch 2/100
320/320 [==============================] - 1s 4ms/step - loss: 0.8182 - accuracy: 0.6995
Epoch 3/100
320/320 [==============================] - 2s 5ms/step - loss: 0.8011 - accuracy: 0.7053
Epoch 4/100
320/320 [==============================] - 1s 4ms/step - loss: 0.7773 - accuracy: 0.7151
Epoch 5/100
320/320 [==============================] - 1s 4ms/step - loss: 0.7490 - accuracy: 0.7261
Epoch 6/100
320/320 [==============================] - 1s 4ms/step - loss: 0.7360 - accuracy: 0.7331
Epoch 7/100
320/320 [==============================] - 1s 4ms/step - loss: 0.7182 - accuracy: 0.7408
Epoch 8/100
320/320 [==============================] - 1s 4ms/step - loss: 0.7029 - accuracy: 0.7472
Epoch 9/100
320/320 [==============================] - 1s 4ms/step - loss: 0.6884 - accuracy: 0.7514
Epoch 10/100
320/320 [==============================] - 1s 4ms/step - loss: 0.6656 - accura

KeyboardInterrupt: ignored